# GPT Action Library (Middleware): AWS Lambda

## 概要

AWS Lambdaは、サーバーレスコンピューティングサービスで、サーバーの管理なしにコードを実行できます。GPT Actionsと組み合わせることで、スケーラブルで費用対効果の高いバックエンド機能を提供できます。

## 主な機能

- **サーバーレス実行**: インフラストラクチャの管理が不要
- **自動スケーリング**: トラフィックに応じて自動的にスケール
- **従量課金**: 実際の使用量に基づく料金体系
- **多言語サポート**: Python、Node.js、Java、C#などをサポート
- **AWSサービス統合**: 他のAWSサービスとのシームレスな連携

## 使用例

### 基本的なLambda関数

```python
import json

def lambda_handler(event, context):
    # GPT Actionからのリクエストを処理
    body = json.loads(event['body'])
    
    # ビジネスロジックを実行
    result = process_request(body)
    
    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
        'body': json.dumps(result)
    }

def process_request(data):
    # カスタムロジックをここに実装
    return {"message": "処理が完了しました", "data": data}
```

## 設定手順

1. **Lambda関数の作成**
   - AWS Lambdaコンソールで新しい関数を作成
   - 適切なランタイムを選択
   - 実行ロールを設定

2. **API Gatewayの設定**
   - Lambda関数にHTTPエンドポイントを提供
   - CORS設定を有効化
   - 認証方法を設定

3. **GPT Actionの設定**
   - OpenAPI仕様を定義
   - Lambda関数のエンドポイントURLを指定
   - 必要に応じて認証情報を設定

## ベストプラクティス

- **エラーハンドリング**: 適切なエラーレスポンスを実装
- **ログ記録**: CloudWatch Logsを活用したデバッグ
- **セキュリティ**: IAMロールによる最小権限の原則
- **パフォーマンス**: コールドスタートの最適化
- **監視**: CloudWatchメトリクスによる監視

## はじめに

この特定のGPT Actionは、**AWS Lambda**関数の構築方法の概要を提供します。このドキュメントは、ユーザーがOAuth保護されたAWS関数をGPT Actionに接続し、サンプルアプリケーションに接続するためのセットアップを支援します。この例では、AWSスタックをセットアップするためにAWS SAM（Serverless Application Model）を使用しています。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの機能を活用してAWS Functionに接続できるようになりました。これにより、AWS内の任意のサービスに接続し、コード/アプリケーションを実行することが可能になります。これは以下のような方法で実現できます：

- AWS Redshift、AWS DynamoDB、AWS S3などのサードパーティサービスへのアクセス、さらに多くのサービスにも対応！
- APIからのテキストレスポンスの前処理が可能（コンテキスト制限の克服、コンテキストやメタデータの追加など）
- サードパーティAPIからテキストを取得する代わりに、ファイルを返すことが可能。これは、データ分析用のCSVファイルを表示したり、PDFファイルを取得してChatGPTにアップロードされたファイルとして扱わせる場合に有用です。


**使用例**: 
- ユーザーがRedshiftでデータを検索する必要があるが、ファイルを返すためにChatGPTとRedshiftの間にミドルウェアアプリが必要な場合（データ分析データの正確性と大量データの処理のため）
- ユーザーがAWS function内で複数のステップを構築しており、ChatGPTを使用してそのプロセスを開始する必要がある場合

## アプリケーション情報と前提条件

AWS Lambdaサービスを活用してミドルウェア関数を作成します。以下のリンクにアクセスして、このスタックに慣れ親しんでください：

- Lambda Website: https://aws.amazon.com/lambda/
- Lambda Documentation: https://docs.aws.amazon.com/lambda/
- AWS SAM docs: https://docs.aws.amazon.com/serverless-application-model/

### 前提条件

開始する前に、以下を作成するアクセス権限を持つAWSコンソールがあることを確認してください：Lambda Function、S3 Buckets、Application Stack、Cognito User Pool、Cognito User Pool App Clients、API Gateway、Lambdaロール、CloudFormationスタック（多く感じるかもしれませんが、これらのサービスの作成は自動化されています！）。

## AWS Lambda関数の作成

AWS Functionを作成するには、AWS SAMを使用できます。SAMテンプレートの例は[こちら](https://github.com/pap-openai/redshift-middleware/blob/main/template.yaml) [0]で確認できます。

このテンプレートには以下が含まれています：
- User PoolとUser Pool Client（OAuth用）
- 認証されたユーザーのみが関数を呼び出せるようにするCognito Authorizer
- Lambda関数を既存のVPCにマッピング（他のAWSサービスへの接続に便利）
- 動的に設定可能なパラメータ（例：認証情報/変数）
- HTTPルートを関数にマッピングするAPI Gateway

このコードは純粋に情報提供目的で、開始時の参考として提供されており、既存のAWSリソースは必要ありません。新しいものを作成するのではなく、既存のuser poolがある場合はそれをマッピングすることをお勧めします。また、他のAWSリソースにアクセスする必要がある場合は、それらのリソースにアクセス可能なVPCでLambdaを設定することをお勧めします。このような設定の例は[RedShift cookbook](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_aws_function)で確認できます。

Cognito Authorizerは、認証されたユーザーのみが関数を呼び出し/アクセスできるようにするために重要なので、環境に合わせて正しく設定してください。

[0]
```
AWSTemplateFormatVersion: '2010-09-09'
Transform: AWS::Serverless-2016-10-31
Description: >
  aws-middleware

  AWS middleware function

Parameters:
  CognitoUserPoolName:
    Type: String
    Default: MyCognitoUserPool
  CognitoUserPoolClientName:
    Type: String
    Default: MyCognitoUserPoolClient

Resources:
  MyCognitoUserPool:
    Type: AWS::Cognito::UserPool
    Properties:
      UserPoolName: !Ref CognitoUserPoolName
      Policies:
        PasswordPolicy:
          MinimumLength: 8
      UsernameAttributes:
        - email
      Schema:
        - AttributeDataType: String
          Name: email
          Required: false

  MyCognitoUserPoolClient:
    Type: AWS::Cognito::UserPoolClient
    Properties:
      UserPoolId: !Ref MyCognitoUserPool
      ClientName: !Ref CognitoUserPoolClientName
      GenerateSecret: true

  MiddlewareApi:
    Type: AWS::Serverless::Api
    Properties:
      StageName: Prod
      Cors: "'*'"
      Auth:
        DefaultAuthorizer: MyCognitoAuthorizer
        Authorizers:
          MyCognitoAuthorizer:
            AuthorizationScopes:
              - openid
              - email
              - profile
            UserPoolArn: !GetAtt MyCognitoUserPool.Arn
        
  MiddlewareFunction:
    Type: AWS::Serverless::Function
    Properties:
      CodeUri: aws-middleware/
      Handler: app.lambda_handler
      Runtime: python3.11
      Timeout: 45
      Architectures:
        - x86_64
      Events:
        SqlStatement:
          Type: Api
          Properties:
            Path: /my_route
            Method: post
            RestApiId: !Ref MiddlewareApi

Outputs:
  MiddlewareApi:
    Description: "API Gateway endpoint URL for Prod stage for SQL Statement function"
    Value: !Sub "https://${MiddlewareApi}.execute-api.${AWS::Region}.amazonaws.com/Prod/my_route"
  MiddlewareFunction:
    Description: "SQL Statement Lambda Function ARN"
    Value: !GetAtt MiddlewareFunction.Arn
  MiddlewareFunctionIamRole:
    Description: "Implicit IAM Role created for SQL Statement function"
    Value: !GetAtt MiddlewareFunctionRole.Arn
  CognitoUserPoolArn:
    Description: "ARN of the Cognito User Pool"
    Value: !GetAtt MyCognitoUserPool.Arn
```

openai-cookbookリポジトリをクローンして、`lambda-middleware`ディレクトリからサンプルのPythonコードとSAMテンプレートを取得できます：

```
git clone https://github.com/pap-openai/aws-lambda-middleware
cd lambda-middleware
```

関数をビルドしてデプロイするには、このディレクトリから以下のコマンドを実行します：

```
sam build
sam deploy --template-file template.yaml --stack-name aws-middleware --capabilities CAPABILITY_IAM
```

デプロイが完了したら、AWS Lambdaでアプリケーションを確認できます：

![../../../images/aws_lambda_1.png](../../../images/aws_lambda_1.png)

認証なしでcurlコマンドを実行することで、認証されていない場合は関数にアクセスできないことを確認できます：

```
curl -d {} <middleware_api_output_url_from_deploy_command>
```

これは`{"message":"Unauthorized"}`を返すはずです。

## AWS Cognitoでの認証設定

_オプション: ユーザープールを作成した場合のみこれらの手順を実行し、既存のものを使用していない場合のみ_

新しく作成したユーザープールにユーザーを作成しましょう。そのために、デプロイコマンドのCognitoUserPoolArnの出力を取得し、"/"の後の値を取得します。これは`your-region_xxxxx`の形式になっているはずです。

```
aws cognito-idp admin-create-user \
    --user-pool-id "your-region_xxxxx" \
    --username johndoe@example.com \
    --user-attributes Name=email,Value=johndoe@example.com \
    --temporary-password "TempPassword123"
```

次に、ログインできるWebページ/ドメインを作成しましょう。AWS Cognitoに移動し、新しく作成したユーザープールを選択して、App Integrationタブに移動します：

![../../../images/aws_lambda_3.png](../../../images/aws_lambda_3.png)

「Domains」をクリックしてから「Create Cognito Domain」をクリックして、Cognitoドメインを作成します。

![../../../images/aws_lambda_8.png](../../../images/aws_lambda_8.png)

ユーザープールのApp IntegrationページでApp client listまでスクロールダウンします：

![../../../images/aws_lambda_9.png](../../../images/aws_lambda_9.png)

アプリクライアントを選択し、Hosted UIを編集します：

![../../../images/aws_lambda_10.png](../../../images/aws_lambda_10.png)

そして、コールバックURL、認証スキーム、OAuthスコープを追加します：

![../../../images/aws_lambda_11.png](../../../images/aws_lambda_11.png)

_ChatGPTがアクションの認証用のコールバックURLを生成する際に、この手順に戻ってくることに注意してください。postman URLは開発目的でのみ使用してください。_

Postmanでこの接続をテストできます。`<api_url>`のAuthorizationで、AWSからclient_id、client_secret、および認証ドメイン用に設定したURLの値をコピー/ペーストし、有効なaccess_tokenを取得するためにスコープに`openid`を追加することを確認してください：

![../../../images/aws_lambda_12.png](../../../images/aws_lambda_12.png)

![../../../images/aws_lambda_13.png](../../../images/aws_lambda_13.png)

取得したaccess_tokenを使用してPostmanでリクエストを実行すると、成功のJSONが返されます：

![../../../images/aws_lambda_14.png](../../../images/aws_lambda_14.png)

## ChatGPTでアクションを作成する

それでは、これをChatGPTに統合しましょう。

アクションを作成し、以下の仕様をコピー＆ペーストしてください：

```
openapi: 3.1.0
info:
  title: Success API
  description: API that returns a success message.
  version: 1.0.0
servers:
  - url: https://3ho5n15aef.execute-api.us-east-1.amazonaws.com/Prod
    description: Main production server
paths:
  /my_route:
    post:
      operationId: postSuccess
      summary: Returns a success message.
      description: Endpoint to check the success status.
      responses:
        '200':
          description: A JSON object indicating success.
          content:
            application/json:
              schema:
                type: object
                properties:
                  success:
                    type: boolean
                    example: true
```

アクションをテストしてみると（「Test」ボタンをクリックできます）、認証されていないため401エラーが表示されます。

それでは、アクションに認証を追加しましょう。

Authentication > OAuthをクリックしてください。
次に、AWS Cognitoの変数を取得する必要があります。User Pool > User Pool App Clientに移動してください。そこからclient IDとclient Secretを取得できます。

![../../../images/aws_lambda_15.png](../../../images/aws_lambda_15.png)

これらの値をChatGPTにコピー＆ペーストしてください。次に、Token URLsを追加しましょう。

User Poolから、以前にホストドメイン用に作成したURLを見つけることができます。

![../../../images/aws_lambda_16.png](../../../images/aws_lambda_16.png)

このURLを取得し、[OAuth用のAWSルート](https://docs.aws.amazon.com/cognito/latest/developerguide/federation-endpoints.html)を追加します。

- token: `<your_url>/oauth2/token`
- authorization: `<your_url>/oauth2/authorize`

これらをChatGPTにコピー＆ペーストしてください。

scopeに`openid`を追加し、Saveをクリックしてください。

## ChatGPT URLでCognitoを設定する

GPT に戻り（アクションのサブビューから移動して）、ChatGPT によって提供された認証用のコールバック URL が表示されます：

![../../../images/aws_lambda_17.png](../../../images/aws_lambda_17.png)

この URL を取得し、User Pool App クライアントのホストされた UI を編集して変更を保存します：

![../../../images/aws_lambda_18.png](../../../images/aws_lambda_18.png)

## 関数のテスト

これで、このアクションを再度テストできます：

![../../../images/aws_lambda_19.png](../../../images/aws_lambda_19.png)

AWS Cognitoページにリダイレクトされ、事前に設定した認証情報を使用してログインできます。

GPTに同じアクションを実行するよう依頼すると、認証が完了してこの関数を実行できるようになったため、正しく応答します！

![../../../images/aws_lambda_20.png](../../../images/aws_lambda_20.png)

# 結論

これで、認証された方法でAWSのアプリケーションと通信できるChatGPTアクションの設定が完了しました！このクックブックでは、ユーザー名/パスワードを使用してCognito Poolを一から作成する方法を示していますが、ニーズに応じてCognitoを設定することをお勧めします（例えば、独自のIDPをCognitoに接続するなど）。

さらに、この関数は他のサービスに接続されていないため、安全な方法でAWS Lambda関数と通信できるという利点があります。そのため、ニーズに合わせてコードとAWS SAMテンプレートを調整することができます。より複雑な関数の例としてRedshiftがあり、これは関数と認証を作成するために同じ手順に従いますが、異なるコード/デプロイメントを持っています。